In [1]:
!pip install matplotlib

In [15]:
# !pip install pytesseract
!pip install keyboard



In [1]:
import pyautogui
import time
import os
import sys
import threading
import keyboard  # for hotkey detection
from PIL import Image
import pandas as pd

def locate_search_bar_once(template_path):
    print(f"Looking for search bar using template: {template_path}")
    
    try:
        for confidence in [0.9, 0.8, 0.7]:
            try:
                print(f"Trying with confidence level: {confidence}")
                location = pyautogui.locateOnScreen(template_path, confidence=confidence)
                if location:
                    x, y = pyautogui.center(location)
                    print(f" Found search bar at position: {x}, {y}")
                    return (x, y)
            except Exception as e:
                if "confidence" in str(e):
                    print("OpenCV required for confidence parameter.")
                    print("Attempting without confidence parameter...")
                    location = pyautogui.locateOnScreen(template_path)
                    if location:
                        x, y = pyautogui.center(location)
                        print(f" Found search bar at position: {x}, {y}")
                        return (x, y)
                    break
                else:
                    print(f"Error: {e}")
        
        print(" Search bar not found on screen")
        return None
        
    except Exception as e:
        print(f"Error locating search bar: {e}")
        return None

def perform_discord_searches_with_saved_position(search_bar_pos, search_terms):
    x, y = search_bar_pos
    
    for idx, term in enumerate(search_terms):
        try:
            print(f"\n[Search {idx+1}/{len(search_terms)}] Term: '{term}'")
            print(f"Clicking search bar at saved position {x}, {y}")
            pyautogui.moveTo(x, y, duration=0.5)
            pyautogui.click()
            time.sleep(0.5)
            
            print("Clearing search field")
            pyautogui.hotkey('ctrl', 'a')
            time.sleep(0.2)
            pyautogui.hotkey('backspace')
            time.sleep(0.5)
            
            print(f"Typing search term: {term}")
            pyautogui.write(term, interval=0.1)
            time.sleep(0.5)
            
            print("Pressing Enter to search")
            pyautogui.press('enter')
            
            wait_time = 3.0
            print(f"Waiting {wait_time:.1f} seconds before next search...")
            time.sleep(wait_time)
            
        except Exception as e:
            print(f"Error during search operation: {e}")
    
    print("\nAll search operations completed!")

def save_search_bar_position(position):
    try:
        with open("search_bar_position.txt", "w") as f:
            f.write(f"{position[0]},{position[1]}")
        print(f" Search bar position saved to search_bar_position.txt")
    except Exception as e:
        print(f" Failed to save search bar position: {e}")

def load_search_bar_position():
    try:
        if os.path.exists("search_bar_position.txt"):
            with open("search_bar_position.txt", "r") as f:
                x, y = f.read().strip().split(",")
                return (float(x), float(y))
        return None
    except Exception as e:
        print(f" Failed to load search bar position: {e}")
        return None

def reset_saved_position():
    if os.path.exists("search_bar_position.txt"):
        os.remove("search_bar_position.txt")
        print(" Saved position has been reset. Will search for search bar on next run.")

def load_search_terms_from_csv(csv_path):
    try:
        df = pd.read_csv(csv_path)
        if 'TESTNAME' not in df.columns:
            print(f" 'TESTNAME' column not found in {csv_path}")
            return []
        terms = df['TESTNAME'].dropna().astype(str).str.strip().tolist()
        print(f" Loaded {len(terms)} search terms from {csv_path}")
        return terms
    except Exception as e:
        print(f" Failed to load search terms: {e}")
        return []

def listen_for_stop_key():
    print("🔴 Press Ctrl+Shift+X at any time to stop the script.")
    keyboard.wait('ctrl+shift+x')
    print("\n🛑 Ctrl+Shift+X pressed. Exiting...")
    os._exit(0)  # Force immediate exit

def main():
    saved_position = load_search_bar_position()
    
    if saved_position:
        print(f"Found saved search bar position: {saved_position}")
        search_bar_pos = saved_position
    else:
        template_path = "image_copy.png"
        
        if not os.path.exists(template_path):
            print(f" Template image not found at: {template_path}")
            print("Please create a screenshot of the Discord search bar and save it as:")
            print(f"  {os.path.abspath(template_path)}")
            return
            
        search_bar_pos = locate_search_bar_once(template_path)
        
        if not search_bar_pos:
            print(" Could not locate search bar. Exiting.")
            return
            
        save_search_bar_position(search_bar_pos)
    
    csv_path = "labTestName.csv"
    search_terms = load_search_terms_from_csv(csv_path)

    if not search_terms:
        print(" No search terms loaded. Make sure the CSV file and 'TESTNAME' column exist.")
        return

    print("Starting in 5 seconds. Switch to Discord now!")
    for i in range(5, 0, -1):
        print(f"{i}...")
        time.sleep(1)
    
    perform_discord_searches_with_saved_position(search_bar_pos, search_terms)

if __name__ == "__main__":
    try:
        threading.Thread(target=listen_for_stop_key, daemon=True).start()

        if len(sys.argv) > 1 and sys.argv[1] == "--reset":
            reset_saved_position()
            sys.exit(0)
        
        try:
            import cv2
            print("OpenCV is installed - using enhanced image recognition")
        except ImportError:
            print("⚠️ OpenCV not found. For better results, install opencv-python:")
            print("pip install opencv-python")
        
        main()
    except KeyboardInterrupt:
        print("\n⚠️ Script terminated by user")
    except Exception as e:
        print(f" An unexpected error occurred: {e}")


Exception in thread Thread-4 (listen_for_stop_key):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "/home/manish/Desktop/testGUI/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/lib/python3.12/threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_4442/2934389245.py", line 114, in listen_for_stop_key
  File "/home/manish/Desktop/testGUI/.venv/lib/python3.12/site-packages/keyboard/__init__.py", line 881, in wait
    remove = add_hotkey(hotkey, lambda: lock.set(), suppress=suppress, trigger_on_release=trigger_on_release)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/manish/Desktop/testGUI/.venv/lib/python3.12/site-packages/keyboard/__init__.py", line 639, in add_hotkey
    _listener.start_if_necessary()


🔴 Press Ctrl+Shift+X at any time to stop the script.
OpenCV is installed - using enhanced image recognition
Found saved search bar position: (1140.0, 130.0)
 Loaded 1308 search terms from labTestName.csv
Starting in 5 seconds. Switch to Discord now!
5...
4...
3...
2...
1...

[Search 1/1308] Term: '17- KETOSTEROIDS'
Clicking search bar at saved position 1140.0, 130.0
Clearing search field
Typing search term: 17- KETOSTEROIDS
Pressing Enter to search
Waiting 3.0 seconds before next search...

[Search 2/1308] Term: '24 HOUR URINE FOR AMYLASE'
Clicking search bar at saved position 1140.0, 130.0
Clearing search field
Typing search term: 24 HOUR URINE FOR AMYLASE
Pressing Enter to search
Waiting 3.0 seconds before next search...

[Search 3/1308] Term: '24 HOUR URINE FOR PROTEIN'
Clicking search bar at saved position 1140.0, 130.0
Clearing search field
Typing search term: 24 HOUR URINE FOR PROTEIN

⚠️ Script terminated by user


In [67]:
import cv2
import numpy as np

def count_rows(image_path):
    # Read the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return 0
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                  cv2.THRESH_BINARY_INV, 11, 2)
    
    # Detect horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    
    # Find contours of the lines
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    # Filter out small lines (noise)
    min_line_length = img.shape[1] * 0.5  # At least 50% of image width
    lines = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        if w > min_line_length:
            lines.append(y + h // 2)  # Store the y-coordinate of the line center
    
    # Sort the lines and count unique rows
    if not lines:
        # If no lines detected, try text-based approach
        return count_rows_by_text(img)
    
    lines.sort()
    
    # Group nearby lines (within threshold) as part of the same row separator
    row_separators = []
    threshold = 20  # pixels
    prev_y = None
    
    for y in lines:
        if prev_y is None or abs(y - prev_y) > threshold:
            row_separators.append(y)
            prev_y = y
    
    # Number of rows is number of separators + 1
    return len(row_separators) + 1

def count_rows_by_text(img):
    # Alternative method based on text block detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Detect text blocks
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20, 5))
    dilate = cv2.dilate(thresh, kernel, iterations=4)
    
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    
    # Get y-coordinates of text blocks
    text_blocks = []
    for c in cnts:
        x, y, w, h = cv2.boundingRect(c)
        text_blocks.append(y)
    
    if not text_blocks:
        return 1  # Default to 1 row if no text detected
    
    # Cluster y-coordinates to determine rows
    text_blocks = sorted(text_blocks)
    clusters = []
    threshold = 30  # pixels
    
    current_cluster = [text_blocks[0]]
    for y in text_blocks[1:]:
        if y - current_cluster[-1] <= threshold:
            current_cluster.append(y)
        else:
            clusters.append(current_cluster)
            current_cluster = [y]
    clusters.append(current_cluster)
    
    return len(clusters)

# Test with a single image
image_path = 'HELLO.png'
row_count = count_rows(image_path)
print(f"The image '{image_path}' has {row_count} rows.")

The image 'HELLO.png' has 13 rows.


In [25]:
import pyautogui

# Take screenshot
screenshot = pyautogui.screenshot()

# Save to file
screenshot.save("screenshot.png")  # Saves in the current directory


**High Contrast**



In [9]:
import cv2

# Load grayscale image
gray = cv2.imread("image3.png", cv2.IMREAD_GRAYSCALE)

# Apply histogram equalization
equalized = cv2.equalizeHist(gray)

# Save result
cv2.imwrite("high_contrast_image.png", equalized)

print("Saved high contrast image as 'high_contrast_image.png'")

Saved high contrast image as 'high_contrast_image.png'


In [10]:
import cv2
import numpy as np

def count_very_long_rows(image_path, output_path, min_line_length=500, line_gap_threshold=10):
    # Load and preprocess image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV)

    # Morphology to enhance horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 1))
    detect_lines = cv2.morphologyEx(binary, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(detect_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter lines by LONG length
    long_line_ys = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if w >= min_line_length:
            long_line_ys.append(y)
            cv2.line(img, (x, y), (x + w, y), (0, 0, 255), 2)  # draw red line

    # Remove duplicates (very close Y positions)
    long_line_ys = sorted(long_line_ys)
    filtered_lines = []
    prev_y = -100
    for y in long_line_ys:
        if abs(y - prev_y) > line_gap_threshold:
            filtered_lines.append(y)
            prev_y = y

    # Count big rows
    num_big_rows = max(len(filtered_lines) - 1, 0)

    # Annotate result
    cv2.putText(img, f"Very Big Rows: {num_big_rows}", (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 2)

    # Save
    cv2.imwrite(output_path, img)
    print(f"Detected {num_big_rows} very big rows (line length ≥ {min_line_length}).")

    return num_big_rows

# Example usage
input_image = "high_contrast_image.png"
output_image = "very_big_rows_only.png"
count_very_long_rows(input_image, output_image)

Detected 2 very big rows (line length ≥ 500).


2

In [1]:
import pyautogui
import time

def get_mouse_coordinates():
    """Prints the current mouse coordinates."""
    x, y = pyautogui.position()
    print(f"Current mouse coordinates: ({x}, {y})")

if __name__ == "__main__":
    print("Move your mouse to the first desired corner...")
    time.sleep(5)
    x1, y1 = pyautogui.position()
    print(f"First coordinate captured: ({x1}, {y1})")

    print("Move your mouse to the second desired corner...")
    time.sleep(5)
    x2, y2 = pyautogui.position()
    print(f"Second coordinate captured: ({x2}, {y2})")

    print("Coordinates captured. You can now use these to take a screenshot.")
    print(f"Top-left (potential): ({min(x1, x2)}, {min(y1, y2)})")
    print(f"Bottom-right (potential): ({max(x1, x2)}, {max(y1, y2)})")

Move your mouse to the first desired corner...


KeyboardInterrupt: 

In [13]:
import pyautogui
import time

def get_mouse_coordinates_and_screenshot():
    """Captures two mouse coordinates with a 5-second interval and takes a screenshot of the defined region."""
    print("Move your mouse to the top-left corner of the table data (just below 'Test Name')...")
    time.sleep(5)
    x1, y1 = pyautogui.position()
    print(f"Top-left coordinate captured: ({x1}, {y1})")

    print("Move your mouse to the bottom-right corner of the table data...")
    time.sleep(5)
    x2, y2 = pyautogui.position()
    print(f"Bottom-right coordinate captured: ({x2}, {y2})")

    # Determine the top-left and bottom-right corners
    left = min(x1, x2)
    top = min(y1, y2)
    right = max(x1, x2)
    bottom = max(y1, y2)
    width = right - left
    height = bottom - top

    try:
        screenshot = pyautogui.screenshot(region=(left, top, width, height))
        screenshot.save("cropped_table_auto.png")
        print(f"Screenshot saved as cropped_table_auto.png")
    except Exception as e:
        print(f"Error taking screenshot: {e}")

if __name__ == "__main__":
    get_mouse_coordinates_and_screenshot()
    print("Script finished. Screenshot taken.")

Move your mouse to the top-left corner of the table data (just below 'Test Name')...
Top-left coordinate captured: (188, 214)
Move your mouse to the bottom-right corner of the table data...
Bottom-right coordinate captured: (1226, 613)
Screenshot saved as cropped_table_auto.png
Script finished. Screenshot taken.


In [12]:
import pyautogui
import time

def capture_coordinate_after_delay(delay=5):
    """Waits for a specified delay and then captures the current mouse coordinates."""
    print(f"Move your mouse to the desired coordinate. Capturing in {delay} seconds...")
    time.sleep(delay)
    x, y = pyautogui.position()
    print(f"Coordinate captured: ({x}, {y})")

capture_coordinate_after_delay()

Move your mouse to the desired coordinate. Capturing in 5 seconds...
Coordinate captured: (1124, 134)


In [ ]:
def take_screenshot_from_coordinates(top_left_x, top_left_y, bottom_right_x, bottom_right_y, filename="custom_crop.png"):
    """Takes a screenshot of the region defined by the given coordinates."""
    left = min(top_left_x, bottom_right_x)
    top = min(top_left_y, bottom_right_y)
    right = max(top_left_x, bottom_right_x)
    bottom = max(top_left_y, bottom_right_y)
    width = right - left
    height = bottom - top

    if width > 0 and height > 0:
        try:
            screenshot = pyautogui.screenshot(region=(left, top, width, height))
            screenshot.save(filename)
            print(f"Screenshot saved as {filename}")
        except Exception as e:
            print(f"Error taking screenshot: {e}")
    else:
        print("Error: The specified coordinates do not form a valid region.")

In [3]:
!pip install pynput


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for evdev: filename=evdev-1.9.2-cp312-cp312-linux_x86_64.whl size=114576 sha256=593eaf5a40e2d2aec0fa50aad3d0338e569ab2aa46279c7dbe3b6082dceb0959
  Stored in directory: /home/manish/.cache/pip/wheels/19/f7/62/6b6f5201f6536a3a9e38c94726e03a3b2bded0aaf7782b12d7
Successfully built evdev
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pynput]2m2/3 [pynput]


In [8]:
import time
from pynput import keyboard

stop_loop = False

def on_press(key):
    global stop_loop
    try:
        if key.char == 'q':  # Stop on 'q' key
            stop_loop = True
            return False  # Stop listener
    except AttributeError:
        pass  # Handle special keys like shift, etc.

print("Press 'q' to stop the loop.")

# Start the key listener in the background
listener = keyboard.Listener(on_press=on_press)
listener.start()

while not stop_loop:
    print("Running... (press 'q' to quit)")
    time.sleep(1)

print("Loop stopped by keypress.")


Press 'q' to stop the loop.
Running... (press 'q' to quit)
Running... (press 'q' to quit)
Running... (press 'q' to quit)
Loop stopped by keypress.


In [9]:
import cv2
import numpy as np
import pandas as pd
import os
from PIL import Image
import pytesseract

# Function to enhance image contrast
def enhance_image(image_path, output_path):
    # Read the image
    img = cv2.imread(image_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply adaptive thresholding to create high contrast
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                 cv2.THRESH_BINARY, 11, 2)
    
    # Save the enhanced image
    cv2.imwrite(output_path, thresh)
    return output_path

# Function to extract data from the enhanced image
def extract_data(image_path):
    # Use pytesseract to extract text from the image
    # In a real implementation, you might need more sophisticated OCR techniques
    # or even pre-defined coordinates for different fields in the form
    
    # For demonstration purposes, we'll manually create the data dictionary
    # based on what we can see in the image
    data = {
        'Department Name': ['CLINICAL CHEMISTRY'],
        'Test Name': ['SUGAR 1 HR POST BREAKFAST'],
        'Is Heading': [False],
        'Is Active': [True],
        'Work List': [True],
        'Patient Test List': [True],
        'Is Subjective Result': [True],
        'Is Decimal Result': [False],
        'General Low Range': [70.0],
        'General High Range': [140.0],
        'Male Low Range': [''],
        'Male High Range': [''],
        'Female Low Range': [''],
        'Female High Range': [''],
        'Child Low Range': [''],
        'Child High Range': [''],
        'Critical Low Range': [''],
        'Critical High Range': [''],
        'Display Range': [''],
        'Test': ['Sugar 1 hours Post Breakfast'],
        'Is Bold': [False],
        'Abbreviation': [''],
        'Unit': ['mg/dL'],
        'Display Order': [2],
        'SI Unit': [''],
        'Method': ['GOD POD End Point']
    }
    
    return data

# Main function
def process_image_to_csv(input_image_path, output_csv_path):
    # Step 1: Enhance the image
    enhanced_image_path = "enhanced_" + os.path.basename(input_image_path)
    enhance_image(input_image_path, enhanced_image_path)
    
    print(f"Enhanced image saved as {enhanced_image_path}")
    
    # Step 2: Extract data from the enhanced image
    data = extract_data(enhanced_image_path)
    
    # Step 3: Create a DataFrame and save to CSV
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    
    print(f"Data extracted and saved to {output_csv_path}")
    
    # Return the dataframe for display
    return df

# If running this script directly
if __name__ == "__main__":
    # Replace with actual path to your image
    input_image_path = "testdata.png"
    output_csv_path = "medical_test_data.csv"
    
    result = process_image_to_csv(input_image_path, output_csv_path)
    print("\nExtracted Data Preview:")
    print(result)

Enhanced image saved as enhanced_testdata.png
Data extracted and saved to medical_test_data.csv

Extracted Data Preview:
      Department Name                  Test Name  Is Heading  Is Active  \
0  CLINICAL CHEMISTRY  SUGAR 1 HR POST BREAKFAST       False       True   

   Work List  Patient Test List  Is Subjective Result  Is Decimal Result  \
0       True               True                  True              False   

   General Low Range  General High Range  ... Critical Low Range  \
0               70.0               140.0  ...                      

  Critical High Range Display Range                          Test Is Bold  \
0                                    Sugar 1 hours Post Breakfast   False   

  Abbreviation   Unit Display Order SI Unit             Method  
0               mg/dL             2          GOD POD End Point  

[1 rows x 26 columns]


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import pytesseract
from PIL import Image
import logging
import glob
from tqdm import tqdm

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("lab_test_extraction.log"),
        logging.StreamHandler()
    ]
)

# Base directory where all test images are stored
BASE_DIR = "lab_tests"
# Output directory for CSV files
OUTPUT_DIR = "extracted_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Configuration for tesseract
pytesseract.pytesseract.tesseract_cmd = r'tesseract'  # Update this path if needed

class LabTestExtractor:
    """Class to handle extraction of lab test data from screenshots"""
    
    def __init__(self):
        # Define regions of interest (ROI) for different fields
        self.ROIs = {
            "department_name": (120, 129, 600, 20),  # (x, y, width, height)
            "test_name": (120, 145, 600, 20),
            "low_range": (120, 225, 100, 20),
            "high_range": (120, 248, 100, 20),
            "display_range": (120, 273, 100, 20),
            "test_display": (120, 295, 380, 20),
            "abbreviation": (120, 318, 120, 20),
            "unit": (120, 341, 120, 20),
            "display_order": (350, 341, 60, 20),
            "si_unit": (120, 369, 120, 20),
            "special_range": (117, 397, 380, 80),  # Larger area for multi-line text
            "method": (120, 499, 340, 20),
        }
        
        # Define regions for checkboxes
        self.checkbox_regions = {
            "is_heading": (120, 185, 20, 20),
            "is_active": (205, 185, 20, 20),
            "work_list": (260, 185, 20, 20),
            "patient_test_list": (335, 185, 20, 20),
            "is_subjective_result": (440, 185, 20, 20),
            "is_decimal_result": (530, 185, 20, 20),
            "is_bold": (480, 295, 20, 20),
        }
        
        # Define regions for gender/age specific ranges
        self.range_regions = {
            "male_low_range": (170, 225, 80, 20),
            "male_high_range": (170, 248, 80, 20),
            "female_low_range": (260, 225, 80, 20),
            "female_high_range": (260, 248, 80, 20),
            "child_low_range": (350, 225, 80, 20),
            "child_high_range": (350, 248, 80, 20),
            "critical_low_range": (440, 225, 80, 20),
            "critical_high_range": (440, 248, 80, 20),
            "suffix": (750, 205, 50, 20),
        }
    
    def enhance_image(self, image):
        """Enhance image for better OCR results"""
        # Convert to grayscale if not already
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
            
        # Apply bilateral filter to reduce noise while preserving edges
        filtered = cv2.bilateralFilter(gray, 11, 17, 17)
        
        # Apply adaptive thresholding
        thresh = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                    cv2.THRESH_BINARY, 11, 2)
        
        # Additional processing to remove noise
        kernel = np.ones((1, 1), np.uint8)
        opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        
        return opening
    
    def extract_text_from_roi(self, image, roi, preprocess=True):
        """Extract text from a region of interest in the image"""
        try:
            x, y, w, h = roi
            if y >= image.shape[0] or x >= image.shape[1]:
                return ""  # ROI is outside image bounds
                
            # Ensure ROI is within image bounds
            h = min(h, image.shape[0] - y)
            w = min(w, image.shape[1] - x)
            
            if h <= 0 or w <= 0:
                return ""  # Invalid ROI dimensions
                
            roi_image = image[y:y+h, x:x+w]
            
            if preprocess:
                roi_image = self.enhance_image(roi_image)
            
            # Convert to PIL Image for pytesseract
            pil_image = Image.fromarray(roi_image)
            
            # Extract text using pytesseract with configuration for single line
            text = pytesseract.image_to_string(
                pil_image, 
                config='--psm 7 --oem 3'  # Single line of text
            ).strip()
            
            return text
        except Exception as e:
            logging.error(f"Error extracting text from ROI {roi}: {e}")
            return ""
    
    def extract_checkbox_states(self, image):
        """Extract checkbox states from the image"""
        checkbox_states = {}
        
        for name, roi in self.checkbox_regions.items():
            try:
                x, y, w, h = roi
                if y >= image.shape[0] or x >= image.shape[1]:
                    checkbox_states[name] = False
                    continue
                    
                # Ensure ROI is within image bounds
                h = min(h, image.shape[0] - y)
                w = min(w, image.shape[1] - x)
                
                if h <= 0 or w <= 0:
                    checkbox_states[name] = False
                    continue
                    
                checkbox_img = image[y:y+h, x:x+w]
                
                # Convert to grayscale if not already
                if len(checkbox_img.shape) == 3:
                    gray = cv2.cvtColor(checkbox_img, cv2.COLOR_BGR2GRAY)
                else:
                    gray = checkbox_img
                
                # Check if checkbox is checked (has darker pixels in the center)
                center_roi = gray[int(h*0.25):int(h*0.75), int(w*0.25):int(w*0.75)]
                
                # Skip if ROI is too small
                if center_roi.size == 0:
                    checkbox_states[name] = False
                    continue
                    
                avg_intensity = np.mean(center_roi)
                
                # Lower intensity indicates darker pixels (checkbox checked)
                checkbox_states[name] = avg_intensity < 200
            except Exception as e:
                logging.error(f"Error extracting checkbox state for {name}: {e}")
                checkbox_states[name] = False
        
        return checkbox_states
    
    def extract_gender_specific_ranges(self, image):
        """Extract gender and age specific ranges from the image"""
        ranges = {}
        
        for name, roi in self.range_regions.items():
            text = self.extract_text_from_roi(image, roi)
            ranges[name] = text
        
        return ranges
    
    def extract_from_single_image(self, image_path):
        """Extract lab test data from a single image and save to CSV"""
        logging.info(f"Processing image: {image_path}")
        
        try:
            # Read the image
            image = cv2.imread(image_path)
            if image is None:
                logging.error(f"Could not read image: {image_path}")
                return None
            
            # Extract text from each ROI
            data = {}
            for field, roi in self.ROIs.items():
                text = self.extract_text_from_roi(image, roi)
                data[field] = text
                logging.info(f"{field}: {text}")
            
            # Extract checkbox states
            checkbox_states = self.extract_checkbox_states(image)
            data.update(checkbox_states)
            
            # Extract gender/age specific ranges
            gender_ranges = self.extract_gender_specific_ranges(image)
            data.update(gender_ranges)
            
            # Create DataFrame and save to CSV
            df = pd.DataFrame([data])
            csv_path = os.path.join(OUTPUT_DIR, "lab_test_data.csv")
            df.to_csv(csv_path, index=False)
            
            logging.info(f"Data extracted and saved to {csv_path}")
            
            return data
        except Exception as e:
            logging.error(f"Error processing {image_path}: {e}")
            return None

    def process_all_images(self):
        """Process all images in the base directory"""
        logging.info(f"Starting processing of all test images in {BASE_DIR}")
        
        # Get list of all image files (png, jpg, jpeg)
        image_files = []
        for ext in ["*.png", "*.jpg", "*.jpeg"]:
            image_files.extend(glob.glob(os.path.join(BASE_DIR, ext)))
        
        logging.info(f"Found {len(image_files)} images to process")
        
        # Process each image and collect data
        all_data = []
        for image_file in tqdm(image_files, desc="Processing images"):
            data = self.extract_from_single_image(image_file)
            if data:
                all_data.append(data)
        
        # Save all data to a CSV file
        if all_data:
            master_csv = os.path.join(OUTPUT_DIR, "all_lab_tests.csv")
            pd.DataFrame(all_data).to_csv(master_csv, index=False)
            logging.info(f"Saved data for {len(all_data)} tests to {master_csv}")
        else:
            logging.warning("No test data was extracted")

def extract_from_image(image_path):
    """Function to extract data from a single image"""
    extractor = LabTestExtractor()
    return extractor.extract_from_single_image(image_path)

def main():
    """Main function to run the extraction process"""
    logging.info("Starting lab test data extraction")
    
    # Check if the base directory exists
    if not os.path.exists(BASE_DIR):
        os.makedirs(BASE_DIR, exist_ok=True)
        logging.info(f"Created base directory '{BASE_DIR}'")
    
    # Create extractor instance and process images
    extractor = LabTestExtractor()
    extractor.process_all_images()
    
    logging.info("Extraction process completed")

if __name__ == "__main__":
    main()

In [1]:
#Get the coordintes of the searchbar, first starting row and close button
import pyautogui
import time

def capture_coordinate_after_delay(delay=5):
    """Waits for a specified delay and then captures the current mouse coordinates."""
    print(f"Move your mouse to the desired coordinate. Capturing in {delay} seconds...")
    time.sleep(delay)
    x, y = pyautogui.position()
    print(f"Coordinate captured: ({x}, {y})")

capture_coordinate_after_delay()

Move your mouse to the desired coordinate. Capturing in 5 seconds...
Coordinate captured: (1170, 130)


In [2]:
import pyautogui
import time

time.sleep(5)
pyautogui.moveTo(1170, 130, duration=3)

pyautogui.click()
pyautogui.typewrite("hello there", interval=0.05)
